In [2]:
#this script will find the revenue by location for the 24 hours before it was run


#bring in relevant libraries
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import numpy as np
import re
import requests
import datetime
from datetime import datetime, timedelta
import time
import httplib, urllib, json, locale
from urlparse import urlparse
import operator
import bson
from collections import OrderedDict
from pymongo import MongoClient
from bson.objectid import ObjectId

In [3]:
merchant_ids = []
#connecting to our db
connection = MongoClient(host =)
db=
merchant = db['Merchants']
for merchants in merchant.find():
     merchant_ids.append(str(merchants['_id']))

In [6]:
for MERCHANT_ID in merchant_ids:
    #eventually we'll want this to be brought in externally from mongo
    #object_id = 'ObjectId(\"' + MERCHANT_ID + '\")'
    doc1 = []
    for doc in merchant.find({'_id':ObjectId(MERCHANT_ID)}):
         doc1.append(doc)
    try:
        access_token = doc1[0]['pos_access_token']
        #print access_token
        record=db['Reports']
        locs = []
    except:
        continue

    #cron = CronTab()
    #job = cron.new(command='/usr/bin/echo')
    #job.day.every(1)


    #!/usr/bin/python
    #
    # Generating a payment report with the Square Connect API.



    # Standard HTTP headers for every Connect API request
    #access_token = 'sq0atp-TkDkbQjLEzdJixG3zo9Lqg'
    request_headers = {'Authorization': 'Bearer ' + access_token,
                       'Accept': 'application/json',
                       'Content-Type': 'application/json'}

    # The base URL for every Connect API request
    connection = httplib.HTTPSConnection('connect.squareup.com')

    # Uses the locale to format currency amounts correctly
    locale.setlocale(locale.LC_ALL, 'en_US')



    # Helper function to convert cent-based money amounts to dollars and cents
    def format_money(amount):
      return locale.currency(amount / 100.)


    # Obtains all of the business's location IDs. Each location has its own collection of payments.
    def get_location_ids():
      request_path = '/v1/me/locations'
      connection.request('GET', request_path, '', request_headers)
      response = connection.getresponse()

      # Transform the JSON array of locations into a Python list
      locations = json.loads(response.read())
      location_ids = []
      for location in locations:
        location_ids.append(location['id'])
        
      return location_ids


    # Downloads all of a business's payments
    def get_payments_day(location_ids):

        #get time right now, timezone should be correct if computer is synced properly
          d = datetime.utcnow()
          diff_time = datetime.utcnow() - timedelta(days = 1)


          #time_now
          # Make sure to URL-encode all parameters

        #turn current time into a bunch of strings that we can add together to form link parameters below
          year,month,day,hour,minute,second = str(d.year),str(d.month),str(d.day),str(d.hour),str(d.minute),str(d.second)
          year_diff,month_diff,day_diff,hour_diff,minute_diff,second_diff = str(diff_time.year),str(diff_time.month),str(diff_time.day),str(diff_time.hour),str(diff_time.minute),str(diff_time.second)

        #if any of our dates/times are only one digit, we add a '0' to the front to make them suitable for URL 
          if len(day) == 1:
                day = '0'+ day
          if len(month) == 1:
                month = '0'+ month
          if len(hour) == 1:
                hour = '0' + hour
          if len(minute) == 1:
                minute = '0' + minute
          if len(second) == 1:
                second = '0' + second

          if len(day_diff) == 1:
                day_diff = '0'+ day_diff
          if len(month_diff) == 1:
                month_diff = '0'+ month_diff
         #####there is a problem here with the days, I think it's when the next day is being called because UTC
          parameters = urllib.urlencode({'begin_time': year_diff + '-' + month_diff + '-' + day_diff + 'T05:00:00',
                                         'end_time': year + '-' + month + '-' + day + 'T05:00:00'})
          payments = []
          # For each location...
          for location_id in location_ids:

            #print 'Downloading payments for location with ID ' + location_id + '...'

            request_path = '/v1/' + location_id + '/payments?' + parameters
            more_results = True

            # ...as long as there are more payments to download from the location...
            while more_results:

              # ...send a GET request to /v1/LOCATION_ID/payments
              connection.request('GET', request_path, '', request_headers)
              response = connection.getresponse()

              # Read the response body JSON into the cumulative list of results
              payments = payments + json.loads(response.read())

              # Check whether pagination information is included in a response header, indicating more results
              pagination_header = response.getheader('link', '')
              if "rel='next'" not in pagination_header:
                more_results = False
              else:

                # Extract the next batch URL from the header.
                #
                # Pagination headers have the following format:
                # <https://connect.squareup.com/v1/LOCATION_ID/payments?batch_token=BATCH_TOKEN>;rel='next'
                # This line extracts the URL from the angle brackets surrounding it.
                next_batch_url = urlparse(pagination_header.split('<')[1].split('>')[0])

                request_path = next_batch_url.path + '?' + next_batch_url.query

          # Remove potential duplicate values from the list of payments
          seen_payment_ids = set()
          unique_payments = []
          loc_data = []
          for payment in payments:
            if payment['id'] in seen_payment_ids:
              continue
            seen_payment_ids.add(payment['id'])
            unique_payments.append(payment)
            loc_data.append(unique_payments)
          return loc_data

    if __name__ == '__main__':

      # Get all payments from all of the business's locations
      payments = get_payments_day(get_location_ids())
      locs = [str(loc) for loc in get_location_ids()]
      pay_data = payments
    
      # Print a sales summary report of the payments
      #print_sales_report(payments)






    #making empty lists to fill with JSON pulled data
    name_list = []
    quant_list = []
    quant_id_list = []
    time_list = []
    location_list = []
    #pulling in item name,quantity purchased for the day, id of item, and time of purchase
    for n in range(len(pay_data)):
        for i in pay_data[n]:
            #all of our entries should be complete every time, if not, we'll add a nan
            try:
                name_list.append(i['itemizations'][n]['name'])
            except:
                name_list.append(np.nan)
            try:
                quant_list.append(str(i['itemizations'][n]['quantity']))
            except:
                quant_list.append(0)
            try:
                quant_id_list.append(i['itemizations'][n]['item_detail']['item_id'])
            except:
                quant_id_list.append(np.nan)
            try:
                time_list.append(i['created_at'])
            except:
                time_list.append(np.nan)
            try:
                location_list.append(i['merchant_id'])
            except:
                location_list.append('')
    # In addition to an Authorization header, requests to the Connect API should
    # include the indicated Accept and Content-Type headers.
    request_headers = {'Authorization': 'Bearer ' + access_token,
                   'Accept':        'application/json',
                   'Content-Type':  'application/json'}

    # Send a GET request to the ListLocations endpoint and obtain the response.
    connection = httplib.HTTPSConnection('connect.squareup.com')
    request_path = '/v1/'+locs[0]+'/items'
    connection.request('GET', request_path, '', request_headers)
    response = connection.getresponse()



    # Convert the returned JSON body into an array of locations you can work with.
    items_data = json.loads(response.read())


    id_list = []
    url_list = []
    names_list = []
    price_list = []
    #pulling in item, id, price of item, and image url

    for m in items_data:
        #try and pull in data, if it doesn't exist for the instance, add a nan to be handled later
        try:      
            names_list.append(m['name'])
            
        except:
            names_list.append(np.nan)
        try:
            id_list.append(m['id'])
        except:
            id_list.append(np.nan)
        try:
            url_list.append(m['master_image']['url'])
        except:
            url_list.append(np.nan)
        try:
            price_list.append(m['variations'][0]['price_money']['amount'])
        except:
            price_list.append(np.nan)
    #make a dataframe of all of the item data
    item_df = pd.DataFrame({'id':id_list,'url':url_list, 'name':names_list, 'price':price_list})
    #make a dataframe of all the payment data
    pay_df = pd.DataFrame({'quantity':quant_list,'id':quant_id_list,'item_name':name_list,'time_of_sale':time_list,'location':location_list})
    #drop entries that are missing item_name and quantity
    pay_df.dropna(subset = ['item_name','quantity']).reset_index(drop = True)

    
    item_df.price=item_df.price/100
    
    #this will allow us to merge price on later
    price_df = item_df[['name','price']]
    #merge item and payment data
    fin_df = item_df.merge(pay_df, left_on = 'name', right_on= 'item_name')

    #make quantity a numeric so we can do math with it
    fin_df.quantity = pd.to_numeric(fin_df.quantity)
    #isolate name, quantity, and location
    revenue_df = fin_df.loc[:,['name','quantity','location']]
    
    #group up name of item, then group by locations within names
    revenue_df = revenue_df.groupby(['name','location']).sum()
    revenue_df = revenue_df.reset_index()
    #merge with price dataframe to bring in price of each item
    revenue_df = revenue_df.merge(price_df, on = 'name')
    #multiply price by quantity to get revenue per product
    revenue_df['revenue'] = revenue_df.quantity * revenue_df.price
    #isolate just location and revenue
    revenue_df = revenue_df[['location','revenue']]
    #group up revenue by sum for each location
    revenue_df = revenue_df.groupby('location').sum()
    revenue_df = revenue_df.reset_index()
    #convert revenue dataframe to a dictionary so we can add it to the mongo
    rev_dict = revenue_df.to_dict(orient = 'list')
    

    try:#for dict_ in rev_dict_list:
        for i,key in enumerate(rev_dict):
            record.update_many({
              'merchant_uid': MERCHANT_ID
            },{
              '$set': 
                  {'Location_revenues':{rev_dict['location'][i]:rev_dict['revenue']}
                  }
            }, upsert=True)
    except:
        pass

    

Empty DataFrame
Columns: [location, quantity, name, price]
Index: []
{'location': [], 'revenue': []}
Empty DataFrame
Columns: [location, quantity, name, price]
Index: []
{'location': [], 'revenue': []}
Empty DataFrame
Columns: [location, quantity, name, price]
Index: []
{'location': [], 'revenue': []}
                name       location  quantity  price
0           Combo #1  A4RYS03VV89MM      25.0  13.75
1           Combo #2  A4RYS03VV89MM       1.0  13.75
2           Combo #3  A4RYS03VV89MM      23.0  13.75
3           Combo #4  A4RYS03VV89MM       1.0  11.75
4      Dipping Sauce  A4RYS03VV89MM       4.0   0.50
5              Drink  A4RYS03VV89MM       5.0   1.50
6       French Fries  A4RYS03VV89MM       6.0   4.00
7       French Fries  A4RYS03VV89MM       6.0   4.00
8   Kimchi Ball 3pcs  A4RYS03VV89MM       4.0   6.50
9       Lg 14pcs KFC  A4RYS03VV89MM       5.0  18.00
10      Med 7pcs KFC  A4RYS03VV89MM      19.0  10.00
11         Pr. Drink  A4RYS03VV89MM       2.0   2.00
12      

In [ ]:
print rev_dict
